# LLM agents คือ

การออกแบบแบบกระบวนทำงาน ของ Large Language Model ให้ทำงานที่มีความซับซ้อนด้วย

In [1]:
import ollama


EMBEDED_MODEL = "all-minilm:latest"
ollama.pull(EMBEDED_MODEL)
_ = [
    print(model["name"])
    for model in ollama.list()["models"]
    if model["name"].startswith("all-minilm")
]


def embed(prompt: str) -> list[float]:
    """Embed a text into a vector.
    """
    return ollama.embeddings(
        model=EMBEDED_MODEL,
        prompt=prompt,
    )["embedding"]


all-minilm:latest


In [22]:
from typing import Callable
import ollama
import json
from retry import retry


LLM_MODEL = "qwen2:7b-instruct-q6_K"


def chat(prompt: str, system_prompt: str, stream: bool = True) -> str:
    response = ollama.chat(
        model="qwen2:7b-instruct-q6_K",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        stream=stream,
    )

    if stream:
        results = ""
        for r in response:
            c = r["message"]["content"]
            print(c, end="")
            results += c
        return results
    else:
        return response["message"]["content"]


class ToolSelectionAgent:
    tools: list[str]
    system_prompt: str = """# Instuction
You are a tool selector agent.
You will be given a user question and candidate tools with index prefix.
You must pick the best tool to answer the question and
return as JSON format with fields "index" determining index of your selected tool
and "arguments" a dictionary object determining arguments to input.
If you can't find any useful, return 0.
"""
    def __init__(self, tools: list[str], model: str = LLM_MODEL):
        """Initialise an agent.

        tools (list[str]): Candidate tools
        model (str): LLM model, default to LLM_MODEL
        """
        self.tools = tools
        self.model = model

    @retry(tries=5, exceptions=AssertionError)
    def select(self, question: str, stream: bool = False):
        """Select tool given a question.

        Args:
            question (str): User's question
        Returns:
            str: Tools selected
        """
        response = chat(system_prompt=self.system_prompt, prompt=question, stream=stream)
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print(f"Cannot parse JSON for {response}")
            raise AssertionError("JSON error")

    
tool_selection_agent = ToolSelectionAgent([])
fucntion_input = tool_selection_agent.select("""# tools
1. search_travel_guide:
Arguments:
keywords (list[str]): A keyword to search.
neightbor (Optional[str]): Specific neighbor can be null

2. search_food_recipe_guide:
Arguments:
keywords (list[str]): A keyword to search

Question:
Recommend some places to have street food in Bangkok
""", stream=False
)
test

{'index': 1,
 'arguments': {'keywords': ['street food', 'Bangkok'], 'neightbor': None}}

In [50]:
# https://github.com/jina-ai/vectordb/
from docarray import BaseDoc
from docarray.typing import NdArray
from docarray import DocList
import numpy as np
import pandas as pd
from vectordb import InMemoryExactNNVectorDB, HNSWVectorDB
import shutil


vector_length = 384
class Movie(BaseDoc):
    title: str = ""
    overview: str = ""
    embedding: NdArray[vector_length]

db = HNSWVectorDB[Movie](workspace="./workspace_path")

In [75]:
from typing import Optional


def search_travel_guide(keywords: list[str], neightbor: Optional[str]) -> str:
    """Search Traval guide database.
    Arguments:
        keywords (list[str]): A keyword to search.
        neightbor (Optional[str]): Specific neighbor can be null
    Returns:
        (str) Travel guide details
    """
    ...
    
def search_food_recipe_guide(keywords: list[str], neightbor: Optional[str]) -> str:
    """Search food recipe.
    Arguments:
        keywords (list[str]): A keyword to search.
    Returns:
        (str) Recipe details
    """
    ...

def search_movie_recommendation(query_phrase: str) -> str:
    """Search movie recommendation.
    Arguments:
        query_phrase (str): Must be string of words, terms, phrases or sentences to input into a vector database.
    Returns:
        (str) Movie details
    """

    query_doc = Movie(embedding=embed(query_phrase))
    search_results = db.search(inputs=DocList[Movie]([query_doc]), limit=5)
    return list(search_results[0].matches)

    def functions_to_prompt(funcs: list[Callable]) -> str:
        prompt = "# Tools\n"
        for i, func in enumerate(funcs, 1):
            func_name = func.__name__
            prompt += f"{i} {func_name}:\n"
            prompt += f"Doc: {func.__doc__}\n"
            prompt += f"Annotation: {func.__annotations__}\n"
            prompt += "-"* 16 + "\n"
        return prompt

class ToolsCollection:
    funcs: list[Callable]
    def __init__(self, funcs: list[Callable]):
        self.funcs = funcs

    def get_prompt(self) -> str:
        funcs = self.funcs
        prompt = "# Tools\n"
        for i, func in enumerate(funcs, 1):
            func_name = func.__name__
            prompt += f"{i} {func_name}:\n"
            prompt += f"Doc: {func.__doc__}\n"
            prompt += f"Annotation: {func.__annotations__}\n"
            prompt += "-"* 16 + "\n"
        return prompt

    def call_function(self, function_index: int, arguments: dict):
        return self.funcs[function_index - 1](**arguments)

tools_collection = ToolsCollection([search_food_recipe_guide, search_travel_guide, search_movie_recommendation])
print(tools_collection.get_prompt())

# Tools
1 search_food_recipe_guide:
Doc: Search food recipe.
    Arguments:
        keywords (list[str]): A keyword to search.
    Returns:
        (str) Recipe details
    
Annotation: {'keywords': list[str], 'neightbor': typing.Optional[str], 'return': <class 'str'>}
----------------
2 search_travel_guide:
Doc: Search Traval guide database.
    Arguments:
        keywords (list[str]): A keyword to search.
        neightbor (Optional[str]): Specific neighbor can be null
    Returns:
        (str) Travel guide details
    
Annotation: {'keywords': list[str], 'neightbor': typing.Optional[str], 'return': <class 'str'>}
----------------
3 search_movie_recommendation:
Doc: Search movie recommendation.
    Arguments:
        query_phrase (str): Must be string of words, terms, phrases or sentences to input into a vector database.
    Returns:
        (str) Movie details
    
Annotation: {'query_phrase': <class 'str'>, 'return': <class 'str'>}
----------------



In [53]:
def get_question_prompt(question: str) -> str:
    return f"""# Question:
{question}
"""

question = "Recommend some places to have street food in Bangkok"
question_prompt = get_question_prompt(question)
print(question_prompt)

# Question:
Recommend some places to have street food in Bangkok



In [80]:
def select_tool(question: str, print_log: bool = False) -> str:
    # Get question prompt
    question_prompt = get_question_prompt(question)

    # Get tools
    if print_log:
        print("Run getting tool")
    available_tools = [
        search_food_recipe_guide,
        search_travel_guide,
        search_movie_recommendation,
    ]
    tools_collection = ToolsCollection(available_tools)
    tools_prompt = tools_collection.get_prompt()

    # Combine prompt
    tool_selection_prompt = tools_prompt + "\n" + question_prompt
    if print_log:
        print(f"Tool selection prompt:\n{tool_selection_prompt}")
    # Select tools
    selected_function = tool_selection_agent.select(tool_selection_prompt)
    if print_log:
        print(f"Function input: {selected_function}")
    tool_results = tools_collection.call_function(
        selected_function["index"], arguments=selected_function["arguments"]
    )
    results = [
        f"Title: {result.title}\nOverview: {result.overview}" for result in tool_results
    ]
    search_result_prompt = "\n----------------\n".join(results)
    if print_log:
        print(f"Getting tool result:")
        print(search_result_prompt)
    
    return search_result_prompt


response = select_tool(
    "Recommend any movies about spy going on a mission", print_log=True
)

Run getting tool
Tool selection prompt:
# Tools
1 search_food_recipe_guide:
Doc: Search food recipe.
    Arguments:
        keywords (list[str]): A keyword to search.
    Returns:
        (str) Recipe details
    
Annotation: {'keywords': list[str], 'neightbor': typing.Optional[str], 'return': <class 'str'>}
----------------
2 search_travel_guide:
Doc: Search Traval guide database.
    Arguments:
        keywords (list[str]): A keyword to search.
        neightbor (Optional[str]): Specific neighbor can be null
    Returns:
        (str) Travel guide details
    
Annotation: {'keywords': list[str], 'neightbor': typing.Optional[str], 'return': <class 'str'>}
----------------
3 search_movie_recommendation:
Doc: Search movie recommendation.
    Arguments:
        query_phrase (str): Must be string of words, terms, phrases or sentences to input into a vector database.
    Returns:
        (str) Movie details
    
Annotation: {'query_phrase': <class 'str'>, 'return': <class 'str'>}
---------

In [87]:
from IPython.display import Markdown


class UserProxyAgent:
    system_prompt: str = "Answer user's question based on given contexts"


    @staticmethod
    def get_prompt(question: str, context: str) -> str:
        prompt = f"""# Context:
{context}

# Question:
{question}
"""
        return prompt

    def response(self, question: str, context: str, stream: bool = False):
        prompt = self.get_prompt(question, context)
        response = chat(system_prompt=self.system_prompt, prompt=prompt, stream=stream)
        return response
    
sample_context = """Title: Act of Valor
Overview: When a covert mission to rescue a kidnapped CIA operative uncovers a chilling plot, an elite, highly trained U.S. SEAL team speeds to hotspots around the globe, racing against the clock to stop a deadly terrorist attack.
----------------
Title: 3 Days to Kill
Overview: A dangerous international spy is determined to give up his high stakes life to finally build a closer relationship with his estranged wife and daughter. But first, he must complete one last mission - even if it means juggling the two toughest assignments yet: hunting down the world's most ruthless terrorist and looking after his teenage daughter for the first time in ten years, while his wife is out of town.
----------------
Title: The Living Daylights
Overview: James Bond helps a Russian General escape into the west. He soon finds out that the KGB wants to kill him for helping the General. A little while later the General is kidnapped from the Secret Service leading 007 to be suspicious.
----------------
Title: Mission: Impossible
Overview: When Ethan Hunt, the leader of a crack espionage team whose perilous operation has gone awry with no explanation, discovers that a mole has penetrated the CIA, he's surprised to learn that he's the No. 1 suspect. To clear his name, Hunt now must ferret out the real double agent and, in the process, even the score.
----------------
Title: Invasion U.S.A.
Overview: A one-man army comes to the rescue of the United States when a spy attempts an invasion."""


question = "Recommend any movies about spy going on a mission"
user_proxy_agent = UserProxyAgent()
response = user_proxy_agent.response(question, sample_context, stream=False)
Markdown(response)

Based on your interest in spy-themed movies with missions at their core, here are some recommendations:

1. **Act of Valor** - This film features an elite U.S. SEAL team engaging in covert operations around the globe to prevent a terrorist attack.

2. **3 Days to Kill** - Although this movie involves family dynamics and personal redemption as its protagonist tries to balance his mission against being a responsible father, it still includes elements of espionage and international spy missions.

3. **The Living Daylights** - A James Bond film that involves complex plots and international intrigue when James Bond helps a Russian General escape the Soviet Union and uncovers a plan by the KGB.

4. **Mission: Impossible** - This series of movies features Ethan Hunt, an agent who leads a team on dangerous missions to complete objectives often involving espionage and counter-terrorism efforts.

5. **Invasion U.S.A.** - A more action-packed thriller where a one-man army fights back against a foreign invasion, making it another suitable choice for fans of spy missions.

Each of these films offers different perspectives on the life of a spy or an agent embarking on critical missions, providing various levels of excitement and depth in storytelling.